In [1]:
import os
import numpy as np
import random
from scipy import ndimage
from skimage import transform
import keras
from keras.layers import Dense, Lambda
from keras.applications import VGG16
from keras.models import Model
from keras.optimizers import SGD
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
import skimage.io as io

# MEAN = np.array([123.0, 117.0, 104.0])
MEAN = np.array([0, 0, 0])
def getModel( output_dim ):
    ''' 
        * output_dim: the number of classes (int)
        
        * return: compiled model (keras.engine.training.Model)
    '''
    vgg_model = VGG16( weights='imagenet', include_top=True )
#     vgg_out = vgg_model.layers[-2].output #Last FC layer's output 
    layer_name = 'fc2'
    x = vgg_model.get_layer(layer_name).output
    temper = 8.0;
    x = Lambda(lambda x: x/temper)(x)
    
    layer_name = 'predictions'
    pred_layer = vgg_model.get_layer(layer_name) 
    weight = pred_layer.get_weights()
    x = Dense(1000, activation='softmax', name='softmax1', weights=weight)(x) 
    
    softmax_layer = Dense(256, activation='softmax')(x) #Create softmax layer taking input as vgg_ou
    #Create new transfer learning model
    tl_model = Model( input=vgg_model.input, output=softmax_layer )

    #Freeze all layers of VGG16 and Compile the model
    #Confirm the model is appropriate
    for l in vgg_model.layers:
        l.trainable = False
        
    layer_name = 'softmax1'
    tl_model.get_layer(layer_name).trainable = False
#     tl_model.get_layer(layer_name).set_weights(weights)
    
    tl_model.summary()

    return tl_model
def convert_image(collect):
#     sz = np.shape(collect)[0]
    sz = len(collect)
    images = np.zeros((sz, 224, 224, 3))
    for i in range(sz):
        img = collect[i]
        img = transform.resize(img, (224, 224))*255
        if (len(img.shape) != 3):
           img = img[:,:,np.newaxis]
        images[i] = img
    return images

sizeOfEachClass = 16
sizeOfTestEachClass = 16
path='/home/yiluo/Homework/253/3/256_ObjectCategories/'
files = os.listdir(path)
X_train, X_val, X_test = [], [], []
Y_train, Y_val, Y_test = [], [], []
label = 0
for category in files:
    sub_path = path + category
    sub_files = os.listdir(sub_path)
    cnt = np.shape(sub_files)[0]
    num = 1
    for img_nm in sub_files:
        if img_nm[-4:] == '.jpg':
            img_path = sub_path + '/' + img_nm
            #img = ndimage.imread(sub_path + '/' + img_nm)
            #img = transform.resize(img, (224, 224))*255
            if num <= np.floor(0.8*cnt):
                if num <= sizeOfEachClass:
                    X_train.append(img_path)
                    Y_train.append(label)
            elif num <= np.floor(0.9*cnt):
                if num <= np.floor(0.8*cnt) + sizeOfTestEachClass:
                    X_val.append(img_path)
                    Y_val.append(label)
            else:
                X_test.append(img_path)
                Y_test.append(label)
            num += 1
    label += 1

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN not available)


In [3]:
#shuffle
ind=range(int(len(X_train)))
random.shuffle(ind)
X_train = [X_train[i] for i in ind]
Y_train = [Y_train[i] for i in ind]
Y_train_label = np.zeros((len(Y_train), 256))
for i in range(len(Y_train)):
    Y_train_label[i, Y_train[i]] = 1
    
#validation  
Y_val_label = np.zeros((len(Y_val), 256))
for i in range(len(Y_val)):
    Y_val_label[i, Y_val[i]] = 1
    
#test
Y_test_label = np.zeros((len(Y_test), 256))
for i in range(len(Y_test)):
    Y_test_label[i, Y_test[i]] = 1

In [6]:
#Output dim for your dataset
output_dim = 256 #For Caltech256

#Data
coll = io.ImageCollection(X_train)
X_train_data = convert_image(coll)
# coll_val = io.ImageCollection(X_val)
# X_val_data = convert_image(coll_val)

In [15]:
#Model
tl_model = getModel( output_dim )
# rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0) # error

# tl_model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

/usr/local/lib/python2.7/dist-packages/Keras-1.2.2-py2.7.egg/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_3 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 4096)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 224, 224, 64)  1792        input_3[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 224, 224, 64)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 112, 112, 64)  0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [17]:
# sgd = keras.optimizers.SGD(lr=2, momentum=0.9, decay=0.0, nesterov=True)
# tl_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

adam = keras.optimizers.Adam(lr=1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
tl_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# adagrad = keras.optimizers.Adagrad(lr=2, epsilon=1e-08, decay=0.0)
# tl_model.compile(loss='categorical_crossentropy', optimizer=adagrad, metrics=['accuracy'])

In [18]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    samplewise_center=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train_data)

In [19]:
hist = tl_model.fit(X_train_data, Y_train_label, 
             batch_size=32, nb_epoch=20, 
             verbose=1, validation_split=0.5,
             shuffle=None, class_weight=None, sample_weight=None)

Train on 2048 samples, validate on 2048 samples
Epoch 1/20
2048/2048 [==============================] - 84s - loss: 7.3957 - acc: 0.0107 - val_loss: 6.7090 - val_acc: 0.0283
Epoch 2/20
2048/2048 [==============================] - 84s - loss: 5.8029 - acc: 0.0522 - val_loss: 5.7863 - val_acc: 0.0693
Epoch 3/20
2048/2048 [==============================] - 83s - loss: 5.0716 - acc: 0.1221 - val_loss: 5.4842 - val_acc: 0.1011
Epoch 4/20
2048/2048 [==============================] - 83s - loss: 4.4759 - acc: 0.1914 - val_loss: 5.1035 - val_acc: 0.1372
Epoch 5/20
2048/2048 [==============================] - 48s - loss: 4.0288 - acc: 0.2495 - val_loss: 4.8052 - val_acc: 0.1602
Epoch 6/20
2048/2048 [==============================] - 45s - loss: 3.6549 - acc: 0.2949 - val_loss: 4.5914 - val_acc: 0.1660
Epoch 7/20
2048/2048 [==============================] - 56s - loss: 3.3784 - acc: 0.3369 - val_loss: 4.4083 - val_acc: 0.2051
Epoch 8/20
2048/2048 [==============================] - 82s - loss: 3.

In [20]:
print(hist.history)

{'acc': [0.0107421875, 0.05224609375, 0.1220703125, 0.19140625, 0.24951171875, 0.294921875, 0.3369140625, 0.37890625, 0.41650390625, 0.45361328125, 0.48095703125, 0.5087890625, 0.53125, 0.56298828125, 0.5791015625, 0.6025390625, 0.615234375, 0.63623046875, 0.6533203125, 0.6708984375], 'loss': [7.3957290276885033, 5.8029070049524307, 5.0716390758752823, 4.4758717082440853, 4.028825905174017, 3.6548697166144848, 3.3783609084784985, 3.1084215752780437, 2.8757450692355633, 2.681011538952589, 2.509543327614665, 2.3504121229052544, 2.2194214202463627, 2.0847434569150209, 1.9879910089075565, 1.8836586568504572, 1.8214411623775959, 1.7181934099644423, 1.6305091688409448, 1.5606490420177579], 'val_acc': [0.0283203125, 0.0693359375, 0.10107421875, 0.13720703125, 0.16015625, 0.166015625, 0.205078125, 0.21875, 0.2216796875, 0.25048828125, 0.26416015625, 0.27294921875, 0.2822265625, 0.2880859375, 0.296875, 0.30126953125, 0.30126953125, 0.30712890625, 0.31787109375, 0.32080078125], 'val_loss': [6.70